# HW Assignment 1, CMOR3 464 INDE 543 - MANUFACTURING PROCESSES AND SYSTEMS 

# Line Setup LP Problem Formulation

## Problem Description

We are tasked with optimizing a production line to meet daily demand under given constraints. The problem includes calculating production with different numbers of workstations, overtime hours, and efficiency levels to determine the most cost-effective setup.

### Data Provided:

#### Costs:
- Unit Material Costs: $30
- Hourly Wage: $60
- Overtime Hourly Wage: $90

#### Current Production:
- Current Production: 315 units/day
- Daily Demand: 420 units/day

#### Other Costs:
- One Time Training Costs: $10,000

### Working Schedule:
- **Workday Start**: 8:00 AM
- **Workday End**: 4:00 PM
- **Overtime Max**: 7:00 PM
- **Normal Work Hours**: 8 hours
- **Maximum Overtime Hours**: 11 hours (including 1 hour break)
- **Break Time**: 1 hour

### Line Setup Information:

| # of Workstations | Overtime (hrs/day) | Capacity (u/day) | Efficiency |
|-------------------|--------------------|------------------|------------|
| 6                 | 0                  |               |        |
| 6                 | 3                  |               |         |
| 7                 | 0                  |                  |            |
| 7                 | 3                  |                  |            |
| 8                 | 0                  |                  |            |
| 8                 | 3                  |                  |            |
| 9                 | 0                  |                  |            |
| 9                 | 3                  |                  |            |

### Activities Data:

| Activities | Time (seconds) | Immediate Predecessors | Workers Allotted |
|------------|----------------|------------------------|------------------|
| 1          | 30             | n/a                    | 1                |
| 2          | 50             | n/a                    | 2                |
| 3          | 40             | 1                      | 1                |
| 4          | 50             | 1                      | 3                |
| 5          | 20             | 2                      | 2                |
| 6          | 10             | 3, 4                   | 4                |
| 7          | 10             | 4, 5                   | 5                |
| 8          | 20             | 2                      | 3                |
| 9          | 10             | 6                      | 4                |
| 10         | 40             | 9                      | 5                |
| 11         | 20             | 7                      | 5                |
| 12         | 30             | 7                      | 7                |
| 13         | 50             | 9                      | 4                |
| 14         | 50             | 10                     | 6                |
| 15         | 10             | 11                     | 6                |
| 16         | 40             | 8, 12                  | 7                |

### Working Hours:

| Metric                         | Value |
|---------------------------------|-------|
| Beginning Work Time             | 8:00 AM |
| End Work Time                   | 4:00 PM |
| Overtime Max                    | 7:00 PM |
| Normal Hours                    | 8 hours |
| Maximum Overtime Hours          | 11 hours |
| Break Time                      | 1 hour |
| Normal Productive Time          | 7 hours |
| Maximal Overtime Productive Time| 10 hours |

## Objective:

The objective is to write Linear Programs (LPs) that optimize the number of workstations. We can use Type 2 Model from the textbook: 

## Type 2 Assembly Line Balancing Model

### Variables:
- $ t_i $: Time required for activity $ A_i $, where $ i = 1, 2, ..., N $
- - $ N = 16 $ for this example
- $ S $: The number of workstations, where $ S = 1, 2, ..., n $
- - $n$ is variable, but it'll be $n \in \{6,7,...,9\}$
- $ C $: Cycle time of the line (maximum time allowed per workstation)
- $ x_{is} $: A binary variable indicating whether activity $ A_i $ is assigned to station $ S $

### Objective:
Minimize cycle time $ C $, i.e., the time required to complete all tasks assigned to a workstation.

$$
\text{Min } C
$$

### Constraints:

1. **Cycle Time Constraint**:  
   The total time assigned to any workstation must not exceed the cycle time $ C $:
   
   $$
   \sum_{i} x_{is} \cdot t_i \leq C, \quad \forall s \in S
   $$

2. **Assignment Constraint**:  
   Each activity must be assigned to exactly one workstation:
   
   $$
   \sum_{s} x_{is} = 1, \quad \forall i \in N
   $$

3. **Precedence Constraint**:  
   Precedence relationships between activities must be respected. If activity $ A_i $ must precede activity $ A_j $, then the assignment must reflect that:
   
   $$
   \sum_{s=1}^{S} x_{is} \cdot s \leq \sum_{s=1}^{S} x_{js} \cdot s, \quad \forall (i, j) \text{ where } A_i \text{ precedes } A_j
   $$


### Problem Summary:
- The goal is to balance the assembly line by minimizing the cycle time while respecting the constraints of workstation assignment and task precedence.
- Each workstation must process tasks in a way that satisfies the overall demand, ensuring efficiency and minimizing idle time.


# Initialize the model
import gurobipy as gp
from gurobipy import GRB

In [224]:
# Dictionary to store time for each job
job_times = {
    1: 30,
    2: 50,
    3: 40,
    4: 50,
    5: 20,
    6: 10,
    7: 10,
    8: 20,
    9: 10,
    10: 40,
    11: 20,
    12: 30,
    13: 50,
    14: 50,
    15: 10,
    16: 40
}


In [225]:
import numpy as np

# Define the precedence dictionary based on your table (activity -> list of immediate predecessors)
precedence_dict = {
    1: [],
    2: [],
    3: [1],
    4: [1],
    5: [2],
    6: [3, 4],
    7: [4, 5],
    8: [2],
    9: [6],
    10: [9],
    11: [7],
    12: [7],
    13: [9],
    14: [10],
    15: [11],
    16: [8, 12]
}

# Initialize an empty precedence matrix with zeros (16x16)
precedence_matrix = np.zeros((num_jobs, num_jobs), dtype=int)

# Fill the precedence matrix based on the precedence_dict
for activity, predecessors in precedence_dict.items():
    for predecessor in predecessors:
        precedence_matrix[activity - 1][predecessor - 1] = 1 

# Print the precedence matrix to verify
print("Precedence Matrix:")
print(precedence_matrix)


Precedence Matrix:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0]]


In [231]:
# Define the number of jobs
num_jobs = 16

# List of different numbers of stations to try
num_stations_list = [6, 7, 8, 9]

# Dictionary to store results for each number of stations
results = {}

for num_stations in num_stations_list:
    print(f"Solving for {num_stations} stations...")
    
    # Create a new Gurobi model
    model = gp.Model("assembly_line_balancing")

    C = model.addVar(name="C")
    model.update()

    model.setObjective(C, GRB.MINIMIZE)

    # Create a matrix of Gurobi binary variables X_{is} for i in [1, 16] and s in [1, num_stations]
    X = [[model.addVar(vtype=GRB.BINARY, name="X_{}_{}".format(i+1, s+1)) 
          for s in range(num_stations)] 
          for i in range(num_jobs)]

    # Add cycle time constraints to the model
    for s in range(num_stations):
        model.addConstr(
            gp.quicksum(X[i][s] * job_times[i+1] for i in range(num_jobs)) <= C,
            name=f"cycle_time_station_{s+1}"
        )

    # Add assignment constraints to the model
    for i in range(num_jobs):
        model.addConstr(
            gp.quicksum(X[i][s] for s in range(num_stations)) == 1,
            name=f"assignment_job_{i+1}"
        )

    # Add precedence constraints to the model
    for i in range(num_jobs):
        for j in range(num_jobs):
            if precedence_matrix[i][j] == 1:
                for k in range(num_stations):
                    lhs = X[i][k]
                    rhs = gp.quicksum(X[j][z] for z in range(k+1))
                    model.addConstr(lhs <= rhs, name=f"precedence_{i+1}_{j+1}_station_{k+1}")

    # Update the model to integrate the new constraints
    model.update()

    # Optimize the model
    model.optimize()

    # Check if the model has an optimal solution
    if model.status == GRB.OPTIMAL:
        # Create a dictionary to store the assignment of activities to stations
        station_assignment = {s+1: [] for s in range(num_stations)}
        
        # Iterate over the variables to get the assignment
        for i in range(num_jobs):
            for s in range(num_stations):
                if X[i][s].x > 0.5:  # If the variable is 1 (assigned)
                    station_assignment[s+1].append(i+1)
        
        # Store the results
        results[num_stations] = {
            'cycle_time': C.x,
            'station_assignment': station_assignment
        }
        
        # Print the assignment
        print(f"Optimal cycle time for {num_stations} stations: {C.x}")
        for station, activities in station_assignment.items():
            print(f"Station {station}: Activities {activities}, Total Time {sum(job_times[activity] for activity in activities)} seconds")
    else:
        print(f"No optimal solution found for {num_stations} stations.")

# Print the results summary
print("\nSummary of results:")
for num_stations, result in results.items():
    print(f"{num_stations} stations: Cycle time = {result['cycle_time']}")

Solving for 6 stations...
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M2 Max
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 124 rows, 97 columns and 657 nonzeros
Model fingerprint: 0xe1af4366
Variable types: 1 continuous, 96 integer (96 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 410.0000000
Presolve removed 17 rows and 0 columns
Presolve time: 0.00s
Presolved: 107 rows, 97 columns, 470 nonzeros
Variable types: 0 continuous, 97 integer (96 binary)

Root relaxation: objective 8.000000e+01, 64 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   80.00000    0   21  

In [233]:
import csv

# Define the file path
output_file_path = '/Users/warrenweissbluth/Documents/CMOR-464-INDE-543-MANUFACTURING-PROCESSES-AND-SYSTEMS/results.csv'

# Write the results to the CSV file
with open(output_file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header
    writer.writerow(["Number of Stations", "Cycle Time", "Station", "Activities", "Total Time (seconds)"])
    
    # Write the data
    for num_stations, result in results.items():
        for station, activities in result['station_assignment'].items():
            total_time = sum(job_times[activity] for activity in activities)
            writer.writerow([num_stations, result['cycle_time'], station, activities, total_time])

print(f"Results written to {output_file_path}")

Results written to /Users/warrenweissbluth/Documents/CMOR-464-INDE-543-MANUFACTURING-PROCESSES-AND-SYSTEMS/results.csv
